# LAB 3: How to setup a project from Scratch

In [ ]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [ ]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b TrainIEMOCAP https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

In [ ]:
!ls

In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

 checkpoints		    README.md
 config.py		    requirements.txt
 data			    SpeechEmotionRecnognition.ipynb
 dataset		   'train IEMOCAP_on_CRNN.py'
'eval IEMOCAP_on_CRNN.py'   train_IEMOCAP_on_CRNN.py
 eval_IEMOCAP_on_CRNN.py    train.py
 eval.py		    utils
 models


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [7]:
!python utils/download_dataset.py

--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/ravdess-emotional-speech-audio
RAVDESS pronto in cache: /kaggle/input/ravdess-emotional-speech-audio
Numero totale di file: 2880

--- Download IEMOCAP ---
Contatto KaggleHub per scaricare: dejolilandry/iemocapfullrelease...
Using Colab cache for faster access to the 'iemocapfullrelease' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/iemocapfullrelease
IEMOCAP pronto in cache: /kaggle/input/iemocapfullrelease
Numero totale di file: 81249

RIEPILOGO DOWNLOAD
RAVDESS: ✅ Successo
IEMOCAP: ✅ Successo

🎉 Tutti i dataset sono stati scaricati con successo!


In [8]:
import os
from pathlib import Path

print("="*80)
print("🔍 RICERCA PERCORSI DATASET")
print("="*80)

# Percorsi possibili dove potrebbero essere i dataset
possible_paths = [
    Path('/kaggle/input/ravdess-emotional-speech-audio'),
    Path.home() / '.cache' / 'kagglehub' / 'datasets',
    Path('/root/.cache/kagglehub/datasets'),
    Path('/tmp/kagglehub/datasets'),
    Path('./data'),
    Path('../data'),
    Path('../../data'),
]

# Aggiungi anche la directory corrente
possible_paths.append(Path.cwd())

print(f"\n📁 Directory corrente: {Path.cwd()}\n")

# Ricerca IEMOCAP
print("🔎 Ricerca IEMOCAP_full_release...")
iemocap_path = None
iemocap_found = False
for base_path in possible_paths:
    if base_path.exists():
        for root, dirs, files in os.walk(base_path):
            if 'IEMOCAP_full_release' in dirs:
                iemocap_path = Path(root) / 'IEMOCAP_full_release'
                print(f"✅ IEMOCAP trovato a: {iemocap_path}")
                iemocap_found = True
                break
    if iemocap_found:
        break

if not iemocap_found:
    print("❌ IEMOCAP non trovato nei percorsi standard")
    iemocap_path = None

# Ricerca RAVDESS
print("\n🔎 Ricerca ravdess-emotional-speech-audio...")
ravdess_path = None
ravdess_found = False
for base_path in possible_paths:
    if base_path.exists():
        for root, dirs, files in os.walk(base_path):
            if 'ravdess-emotional-speech-audio' in dirs:
                ravdess_path = Path(root) / 'ravdess-emotional-speech-audio'
                print(f"✅ RAVDESS trovato a: {ravdess_path}")
                ravdess_found = True
                break
    if ravdess_found:
        break

if not ravdess_found:
    print("❌ RAVDESS non trovato nei percorsi standard")
    ravdess_path = None

# Lista contenuti della directory data/ se esiste
print("\n📂 Contenuto della cartella 'data/' (se presente):")
data_dir = Path('./data')
if data_dir.exists():
    for item in data_dir.iterdir():
        print(f"   - {item.name}")
else:
    print("   ❌ Cartella 'data/' non trovata")

print("\n" + "="*80)
print("✅ VARIABILI SALVATE:")
print(f"   - iemocap_path = {iemocap_path}")
print(f"   - ravdess_path = {ravdess_path}")
print("="*80)

🔍 RICERCA PERCORSI DATASET

📁 Directory corrente: /content/speech-emotion-recognition-25

🔎 Ricerca IEMOCAP_full_release...
✅ IEMOCAP trovato a: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release

🔎 Ricerca ravdess-emotional-speech-audio...
✅ RAVDESS trovato a: /root/.cache/kagglehub/datasets/uwrfkaggler/ravdess-emotional-speech-audio

📂 Contenuto della cartella 'data/' (se presente):
   - .gitkeep

✅ VARIABILI SALVATE:
   - iemocap_path = /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release
   - ravdess_path = /root/.cache/kagglehub/datasets/uwrfkaggler/ravdess-emotional-speech-audio


In [9]:
from torch.utils.data import DataLoader
from dataset.custom_iemocap_dataset import CustomIEMOCAPDataset
from dataset.custom_ravdess_dataset import CustomRAVDESSDataset

print(f"////////////////////////////////////////////////////////////////////////////////////////////")
print(f"Dataset IEMOCAP")
print(f"////////////////////////////////////////////////////////////////////////////////////////////")

# Usa il percorso trovato in precedenza, altrimenti fallback
if iemocap_path and iemocap_path.exists():
    dataset_IEMOCAP_path = str(iemocap_path)
    print(f"✅ Usando percorso trovato: {dataset_IEMOCAP_path}")
else:
    dataset_IEMOCAP_path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release'
    print(f"⚠️  Percorso non trovato, usando fallback: {dataset_IEMOCAP_path}")

# Create IEMOCAPdatasets
train_IEMOCAP_dataset = CustomIEMOCAPDataset(dataset_root=dataset_IEMOCAP_path, split='train')
val_IEMOCAP_dataset = CustomIEMOCAPDataset(dataset_root=dataset_IEMOCAP_path, split='validation')
test_IEMOCAP_dataset = CustomIEMOCAPDataset(dataset_root=dataset_IEMOCAP_path, split='test')

print(f"Train samples: {len(train_IEMOCAP_dataset)}")
print(f"Val samples: {len(val_IEMOCAP_dataset)}")
print(f"Test samples: {len(test_IEMOCAP_dataset)}")

# Create IEMOCAP DataLoaders
batch_size = 4
train_IEMOCAP_dataloader = DataLoader(train_IEMOCAP_dataset, batch_size=batch_size, shuffle=True)
val_IEMOCAP_dataloader = DataLoader(val_IEMOCAP_dataset, batch_size=batch_size, shuffle=False)
test_IEMOCAP_dataloader = DataLoader(test_IEMOCAP_dataset, batch_size=batch_size, shuffle=False)


print(f"////////////////////////////////////////////////////////////////////////////////////////////")
print(f"Dataset RAVDESS")
print(f"////////////////////////////////////////////////////////////////////////////////////////////")

# Usa il percorso trovato in precedenza, altrimenti fallback
if ravdess_path and ravdess_path.exists():
    dataset_RAVDESS_path = str(ravdess_path)
    print(f"✅ Usando percorso trovato: {dataset_RAVDESS_path}")
else:
    dataset_RAVDESS_path = '/kaggle/input/ravdess-emotional-speech-audio'
    print(f"⚠️  Percorso non trovato, usando fallback: {dataset_RAVDESS_path}")

# Create RAVDESS datasets
train_RAVDESS_dataset = CustomRAVDESSDataset(dataset_root=dataset_RAVDESS_path, split='train')
val_RAVDESS_dataset = CustomRAVDESSDataset(dataset_root=dataset_RAVDESS_path, split='validation')
test_RAVDESS_dataset = CustomRAVDESSDataset(dataset_root=dataset_RAVDESS_path, split='test')

print(f"Train samples: {len(train_RAVDESS_dataset)}")
print(f"Val samples: {len(val_RAVDESS_dataset)}")
print(f"Test samples: {len(test_RAVDESS_dataset)}")

# Create RAVDESS DataLoaders
batch_size = 4
train_RAVDESS_dataloader = DataLoader(train_RAVDESS_dataset, batch_size=batch_size, shuffle=True)
val_RAVDESS_dataloader = DataLoader(val_RAVDESS_dataset, batch_size=batch_size, shuffle=False)
test_RAVDESS_dataloader = DataLoader(test_RAVDESS_dataset, batch_size=batch_size, shuffle=False)

////////////////////////////////////////////////////////////////////////////////////////////
Dataset IEMOCAP
////////////////////////////////////////////////////////////////////////////////////////////
✅ Usando percorso trovato: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release
✅ Caricate 5531 etichette
🔍 Raccogliendo campioni audio...
✅ Raccolti 2943 campioni audio
   - Solo campioni improvvisati
   - Emozioni: ['neutral', 'happy', 'sad', 'angry', 'happy']
📊 Statistiche del dataset IEMOCAP:

📊 ANALISI IEMOCAP TRAINING SET

🔹 SAMPLES TOTALI: 1678
🔹 SESSIONI: ['1', '2', '3']
🔹 SPEAKER UNICI (session, gender): 6
   Elenco: [('1', 'F'), ('1', 'M'), ('2', 'F'), ('2', 'M'), ('3', 'F'), ('3', 'M')]
🔹 IMPROVVISAZIONI UNICHE: 8

👥 SPEAKER INDEPENDENCE (per verificare leakage):
   - Sessione 1: (Ses1, F), (Ses1, M)
   - Sessione 2: (Ses2, F), (Ses2, M)
   - Sessione 3: (Ses3, F), (Ses3, M)

🎭 DISTRIBUZIONE EMOZIONI:
   - Angry     :  174 ( 10.4%) ██

# Step 4: Train your model and visualize training

In [10]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [11]:
#!python train.py
!python train_IEMOCAP_on_CRNN.py

✓ Classe SimpleEarlyStopping pronta!
Using device: cpu
Dataset: IEMOCAP

✅ IEMOCAP trovato: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release

📊 Caricamento IEMOCAP dataset...
✅ Caricate 5531 etichette
🔍 Raccogliendo campioni audio...
✅ Raccolti 2943 campioni audio
   - Solo campioni improvvisati
   - Emozioni: ['neutral', 'happy', 'sad', 'angry', 'happy']
📊 Statistiche del dataset IEMOCAP:

📊 ANALISI IEMOCAP TRAINING SET

🔹 SAMPLES TOTALI: 1678
🔹 SESSIONI: ['1', '2', '3']
🔹 SPEAKER UNICI (session, gender): 6
   Elenco: [('1', 'F'), ('1', 'M'), ('2', 'F'), ('2', 'M'), ('3', 'F'), ('3', 'M')]
🔹 IMPROVVISAZIONI UNICHE: 8

👥 SPEAKER INDEPENDENCE (per verificare leakage):
   - Sessione 1: (Ses1, F), (Ses1, M)
   - Sessione 2: (Ses2, F), (Ses2, M)
   - Sessione 3: (Ses3, F), (Ses3, M)

🎭 DISTRIBUZIONE EMOZIONI:
   - Angry     :  174 ( 10.4%) ██
   - Happy     :  472 ( 28.1%) █████
   - Neutral   :  638 ( 38.0%) ███████
   - Sad       :  394 ( 23

# Step 5: Evaluate your model

1.   List item
2.   List item



In [12]:
#!python eval.py
!python eval_IEMOCAP_on_CRNN.py

Using device: cpu
Dataset: IEMOCAP

✅ IEMOCAP trovato: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1/IEMOCAP_full_release

Loading IEMOCAP test set...
✅ Caricate 5531 etichette
🔍 Raccogliendo campioni audio...
✅ Raccolti 2943 campioni audio
   - Solo campioni improvvisati
   - Emozioni: ['neutral', 'happy', 'sad', 'angry', 'happy']
📊 Statistiche del dataset IEMOCAP:

📊 ANALISI IEMOCAP TEST SET

🔹 SAMPLES TOTALI: 731
🔹 SESSIONI: ['5']
🔹 SPEAKER UNICI (session, gender): 2
   Elenco: [('5', 'F'), ('5', 'M')]
🔹 IMPROVVISAZIONI UNICHE: 8

👥 SPEAKER INDEPENDENCE (per verificare leakage):
   - Sessione 5: (Ses5, F), (Ses5, M)

🎭 DISTRIBUZIONE EMOZIONI:
   - Angry     :   31 (  4.2%) 
   - Happy     :  280 ( 38.3%) ███████
   - Neutral   :  287 ( 39.3%) ███████
   - Sad       :  133 ( 18.2%) ███

📋 DISTRIBUZIONE CAMPIONI PER SESSIONE:
   - Sessione 5:  731 (100.0%) ████████████████████
      └─ Angry     :  31 (  4.2%)
      └─ Happy     : 280 ( 38.3%)
      └─ Neu